<a href="https://colab.research.google.com/github/Viroslav/SBERMEDAI/blob/main/SbermedAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import torch
import random
#import numpy as np
import os

torch.backends.cudnn.deterministic = True
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision import datasets 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision
from skimage import io
import pandas as pd
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score



# fix random seed
random.seed(0)
#np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

# using cuda
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
class SitNotSitDataset(Dataset):
    def __init__(self, xlsx_file, root_dir, transform = None):
        self.annotations = pd.read_excel(xlsx_file, index_col=0)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        
        if self.transform:
            image = self.transform(image)
            
        return(image, y_label)

In [3]:
dataset = SitNotSitDataset(xlsx_file = '/content/Images/labels2.xlsx',
                          root_dir = '/content/Images',
                          transform = transforms.ToTensor())

In [4]:
len(dataset)

43

In [5]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

In [6]:
batch_size = 1

In [7]:
train_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_set, batch_size = batch_size, shuffle = True)

In [8]:
train_loader

## Using GoogleNet

In [ ]:
model = torchvision.models.googlenet(pretrained = True)

In [ ]:
model.to(device)

## Optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

In [17]:
num_epochs = 5

## Traininng

In [18]:
for epoch in range(num_epochs):
    losses = []
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device = device)
        targets = targets.to(device = device)
        
        #forward
        scores = model(data)
        loss = criterion(scores,targets)
        
        losses.append(loss.item())
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #adam step
        optimizer.step()
    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

Cost at epoch 0 is 0.7226389929652214
Cost at epoch 1 is 0.6702830205361049
Cost at epoch 2 is 0.5239830849071344
Cost at epoch 3 is 0.5042044512927533
Cost at epoch 4 is 0.38671358476082485


## Accuracy

In [75]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    y_true = []
    y_score = []
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            y_true.append(y.item())
            y_score.append(predictions.item())
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}", '\t',  
              f"and roc_auc_score: {roc_auc_score(y_true, y_score)} ")

    model.train()

In [76]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

Checking accuracy on Training Set
Got 18 / 30 with accuracy 60.0 	 and roc_auc_score: 0.44886363636363635 


In [77]:
print("Checking accuracy on Testing Set")
check_accuracy(test_loader, model)

Checking accuracy on Testing Set
Got 11 / 13 with accuracy 84.61538461538461 	 and roc_auc_score: 0.6666666666666667 
